# FTGP Week 2: Blockchain fundamentals

This week, we will use the cryptographic primitives we saw last week to build our own blockchain!

Understanding how a typical blockchain works is a key part of understanding the benefits (and drawbacks) this technology brings. Knowing how the underling systems works, will also improve your ability to build financial applications on top of it.

We will be using Python and the packages we used last week, plus you might need the `cbor` (https://cbor2.readthedocs.io/en/latest/) package.

***Note:*** The following worksheet requires good programming skills. However, the aim of the worksheet is not to teach you how to code, but to allow you to build your own blockchain system. If you are not feeling confident in your programming abilities, you might wish to use a non-programming alternative, ETH.build (https://sandbox.eth.build/). A series of helpful videos (https://www.youtube.com/playlist?list=PLJz1HruEnenCXH7KW7wBCEBnBLOVkiqIi) is also available to guide you throughout the process.


### Question 1: Transactions

Design and implement a `Transaction` class that includes the following fields

- sender's public key
- receiver's public key
- transaction amount (integer >0)
- comment (arbitrary text, can be empty)
- sender's signature (signing the transaction data)

Your class should also implement the following methods:

- new: to create a new transaction
- sign: to sign the transaction data and produce a valid signature
- validate: to validate a transaction

The following skeleton code might be helpful.

In [34]:
import ecdsa
import cbor2

class Transaction():

    # The init function is called when this class is instantiated
    def __init__(self):
        self.data = {
            "sender" : None,
            "receiver" : None,
            "amount" : None,
            "comment" : None,
        }
    
    # The __eq__ function is called when comparing two objects of this class
    def __eq__(self, check):
        if ( (self.data["sender"] == check.data["sender"]) and (self.data["receiver"] == check.data["receiver"]) and (self.data["amount"] == check.data["amount"]) and (self.data["comment"] == check.data["comment"]) ):
            return True
        else:
            return False
        
    # Serializes data to JSON string
    def serialize(self):
        return cbor2.dumps(self.data)
    
    # Deserializes object from JSON string
    def deserialize(self, obj):
        self.data = cbor2.loads(obj)
        
    def new(self, sender_public_key, receiver_public_key, amount, comment = ""):
        # Create transaction from passed values
        # Hint: store the keys as byte strings

        self.data["sender"]    = sender_public_key.to_string()
        self.data["receiver"]  = receiver_public_key.to_string()
        self.data["amount"]    = amount
        self.data["comment"]   = comment

    def sign(self, sender_private_key):
        # Sign the transaction with the private key

        signature = sender_private_key.sign(self.serialize())
        return signature

    def validate(self, sender_public_key, signature):
        # Validate transaction correctness (i.e. valid signature)
        # Hint: verify the signature of the transaction hash

        return sender_public_key.verify(signature, self.serialize())

Use the following code to test your implementation.

In [51]:
# create sender public/private key
pr_sender = ecdsa.SigningKey.generate()
pub_sender = pr_sender.verifying_key

# create receiver secret/private key
pr_receiver = ecdsa.SigningKey.generate()
pub_receiver = pr_receiver.verifying_key

# create transaction
test = Transaction()
test.new(pub_sender, pub_receiver, 42)

# serialize transaction
serialized = test.serialize()

# create second transaction
deserialized = Transaction()
# deserialize original transaction
deserialized.deserialize(serialized)

# check whether transactions are the same
assert test == deserialized, "Transactions are not the same"

# sign transaction
signature = test.sign(pr_sender)

# validate the signature
assert test.validate(pub_sender, signature), "Signature is not valid"

### Question 2: Block

For this question we will need the Merkle tree we implemented last week. An implementation is provided below, but feel free to use your own implementation from last week.

In [38]:
import hashlib
import math

class MerkleTree():

    def __init__(self):
        self.leaves = [] # entries are assumed to be bytes
        self.hashes = []
        self.root = None

    def add(self, entry):
        # Add entries to tree 
        
        self.leaves += [entry]
        self.hashes += [hashlib.sha256(entry).digest()]

    def build(self):
        # Build tree computing new root

        level = math.ceil(math.log(len(self.hashes), 2)) # next power of two
        if 2**level != len(self.hashes): # if not complete tree
            for i in range(2**level - len(self.hashes)): # add empty leaves so that the tree is complete
                self.add(b"")

        for i in range((2**level)-1): # calculate the rest of the blocks
            self.hashes += [hashlib.sha256(self.hashes[2*i] + self.hashes[2*i+1]).digest()]
        
        self.root = self.hashes[-1] # set root

    def get_proof(self, entry):
        # Get membership proof for a given entry

        if entry not in self.leaves:
            return None
        else:
            degree = math.log(len(self.leaves), 2)
            index = self.leaves.index(entry)
            proof = []
            while self.hashes[index] != self.root:
                if index % 2 == 0:
                    proof += [(self.hashes[index+1], 'r')] # add right sibling
                else:
                    proof += [(self.hashes[index-1], 'l')] # add left sibling
                index = int(2**degree + math.floor(index/2)) # go to parent
            return proof

    def get_root(self):
        # Return the current root

        if self.root == None:
            return b"0"
        else:
            return self.root

Design and implement a `Block` class. Each `Block` object should have:

- set of serialized transactions that form a Merkle tree
- header that includes:
    - block number
    - hash of the previous block's header
    - root of the hash tree
    - timestamp (Unix timestamp expressed as an integer)
    - nonce (a random number needed to generate PoW)
- methods:
    - new: Create a new block from passed values
    - get_hash: Return the hash of the provided values
    - addNonce: Add nonce to block header
    - serialize: Serializes object to CBOR or JSON string

To make this a bit easier, at this point you don't have to implement any checks for the validity of the included transactions.

In [48]:
class Block():

    def __init__(self):
        self.tree = None
        self.header = {
            "number"    : None,
            "prev_hash" : None,
            "root_hash" : None,
            "timestamp" : None,
            "nonce"     : None
        }

    def serialize(self):
        # Serializes object to CBOR
        
        return cbor2.dumps({"transactions": self.tree.leaves, "number": self.header["number"], "prev_hash": self.header['prev_hash'], "nonce": self.header['nonce'], "time": self.header['timestamp']})
    
    def deserialize(self, obj):
        # Deserializes object from CBOR
        
        data = cbor2.loads(obj)
        
        self.new(data["transactions"], data["number"], data["prev_hash"], data["time"], data["nonce"])

    def new(self, transactions, number, prev_hash, timestamp, nonce):
        # Create a new block by adding the transactions to the merkle tree and updating the header
        # Hint: you can assume that the transactions are already serialized
        
        merkle_tree = MerkleTree()
        for tx in transactions:
            merkle_tree.add(tx)
        merkle_tree.build()

        self.tree = merkle_tree
        self.header["number"] = number
        self.header["prev_hash"] = prev_hash
        self.header["root_hash"] = merkle_tree.get_root()
        self.header["timestamp"] = timestamp        
        self.header["nonce"] = nonce
       
    def block_hash(self):
        # Return the hash of the block header
        # The format should be something like: hash = H(number|prev_hash|root|time|nonce)

        hash = hashlib.sha256()
        hash.update(bytes(self.header["number"]))
        hash.update(self.header["prev_hash"])
        hash.update(self.header["root_hash"])
        hash.update(bytes(self.header["timestamp"]))
        hash.update(bytes(self.header["nonce"]))
        return hash.digest()

Use the following code to test your implementation:

In [52]:
import time

txs = []
for i in range(8):

    sk_sender = ecdsa.SigningKey.generate()
    vk_sender = sk_sender.verifying_key
    sk_receiver = ecdsa.SigningKey.generate()
    vk_receiver = sk_receiver.verifying_key

    tr = Transaction()
    tr.new(vk_sender, vk_receiver, i)
    tr.sign(sk_sender)
    serialized = tr.serialize()

    # it was assumed in the implementation that the transactions are already serialized
    txs.append(serialized)

block = Block()
block.new(txs, 5, b"0", int(time.time()), 42)

serialized = block.serialize()

deserialized = Block()
deserialized.deserialize(serialized)

assert block.block_hash() == deserialized.block_hash(), "Blocks are not the same"

### Question 3: 

Design and implement the `Blockchain` class. A `Blockchain` object contains `Block` object(s). Each `Blockchain` object should:

- store the genesis block
- keep track of all blocks
- methods:
    - add: Add a new block to the blockchain
    - last_hash: Returns the hash of the latest block
    - last_number: Returns the number of the latest block

In [53]:
class Blockchain():

    def __init__(self):
        self.head = None
        self.blocks = []
    
    def add(self, block):
        # Add a new block to the chain

        self.blocks.append(block)
        if self.head == None:
            # this is the genesis block
            self.head = block

    def last_hash(self):
        # Return the hash of the last block

        last = self.blocks[-1]
        hash = last.block_hash()
        return hash

    def last_number(self):
        # Return the number of the last block
        
        last = self.blocks[-1]
        num = last.header["number"]
        return num

Use the following code to test your implementation:

In [56]:
txs = []
for i in range(8):

    sk_sender = ecdsa.SigningKey.generate()
    vk_sender = sk_sender.verifying_key
    sk_receiver = ecdsa.SigningKey.generate()
    vk_receiver = sk_receiver.verifying_key

    tr = Transaction()
    tr.new(vk_sender, vk_receiver, i)
    tr.sign(sk_sender)
    serialized = tr.serialize()

    txs.append(serialized)

block = Block()
block.new(txs, 123, b"0", int(time.time()), 456)
hash = block.block_hash()
#print(hash)

blockchain = Blockchain()
blockchain.add(block)
#print(blockchain.last_hash())

assert hash == blockchain.last_hash(), "Hashes are not the same"

### Bonus Question 4:

Congratulations! If you have reached this point, you have implemented your own basic blockchain system!

However, at the current state, our blockchain is missing a number of features. Introduce:
- The Coinbase transaction. Introduce a new special transaction, where the miner who found a new block is rewarded with 100 coins. Make sure that only one such transaction can be included per block.
- PoW. The hash of every new block's header should be less than `TARGET` (a global parameter, set now to `\x00\x0f\xff\xff\xff\xff...\xff`). You also need to implement a function that finds the appropriate block nonce for each block.
- Forks and their handling. Modify your implementation, such that `add()` allows to anchor a new block to any given existing block. Implement the `resolve()` method, that returns the longest chain (e.g., it can return the latest block of the longest chain). 
- At the moment, our blockchain system doesn't perform any checks on the transactions. Modify your block implementation so that each block contains a record of user accounts (i.e. mapping from public keys to balances). Then create a method that makes sure that a transaction is valid (given the block's user record) before adding it to the block. Otherwise the transaction should be discarded without any state changes. Also add a method to your blockchain class to validate that the record of each block is consistent. Make sure that the accounts are matching the ones in the previous block and that the transactions of the block are executed correctly.
- Add checks to your blockchain class to make sure that any block that is added to the blockchain is valid.
- At the moment, transactions can be repeated. Add a counter associated with each account that makes each transaction unique. Add checks to your block class to make sure this is enforced.

**Potential Hints:**
- The Coinbase transaction. Introduce a new special transaction, where the miner who found a new block is rewarded with 100 coins. 
    - This can be achieved by adding a new function to the transaction class. Make sure that you select an appropriate sender (e.g 0x0 or None) and that you account for this is your validation method.
- Make sure that only one such transaction can be included per block.
    - This is a bit more tricky. You need to add a check in your blockchain class that makes sure that only one transaction is the coinbase (a simple boolean would work). You can distinguish the coinbase transaction as it will have a special sender.
- PoW. The hash of every new block's header should be less than `TARGET` (a global parameter, set now to `\x00\x0f\xff\xff\xff\xff...\xff`). 
    - You can easily check this by adding a check in your blockchain class that checks that the hash is less than the `TARGET`.
- Write a function that finds the appropriate block nonce for PoW.
    - This should just be a simple function that tries many nonces (using the `get_hash` method) and returns the first nonce that works.
- Forks and their handling. Modify your implementation, such that `add()` allows to anchor a new block to any given existing block. 
    - This is again tricky. I think the easiest way to achieve this is to have a field in each block that points to its parent block. This field will be populated in the blockchain class. Finally you can keep a list of all the endpoints of forks in your blockchain class. If a new block doesn't attach to one of the endpoints, then starting with each endpoint, you can go back (to its parent) until you find the block that it attaches to. In this case, you have a fork so you need to include this new block in your list of endpoints. 
- Implement the `resolve()` method, that returns the longest chain (e.g., it can return the latest block of the longest chain). 
    - This can be done easily by checking all the endpoints of forks and selecting the one with the highest block number.
- At the moment, our blockchain system doesn't perform any checks on the transactions (apart from verifying the signature). Modify your block implementation so that each block contains a record of user accounts (i.e. mapping from public keys to balances).
    - This can be done by adding a `balances` dictionary in every block. Then when a new block is created, you copy the balances from the previous block and use that as your starting point.
- Then create a method that makes sure that a transaction is valid (given the block's user record) before adding it to the block. Otherwise the transaction should be discarded without any state changes.
    - For each transaction that you are adding to the block, make sure that the sender has enough coins. Then change the balances of the sender and receiver. After you process all the transactions, you should end up with the new balances dictionary that you can attach to your new block.
- At the moment, transactions can be repeated. Add a counter associated with each account that makes each transaction unique.
    - This is similar to the balances. Add a `counts` dictionary in each Block that keeps a counter for every sender. Then modify the transaction class so that a counter field is included. When creating a new transaction make sure to get the counter from the previous block for every sender and include it in your transaction.
- Finally, add a method to your blockchain class to validate that the accounts (balances and counters) of each block. Make sure that the accounts and matching the ones in the previous block after the transactions of the block.
    - This should validate the previous two dictionaries. Starting with the values in the previous block, you can update the balances and counters with each transaction that's included in the new block and then at the end, check if the dictionaries you have created match the ones present in the new block.